**合併數據**

In [2]:
import pandas as pd
import glob
import os

folder_path = './Training data/'
files = glob.glob(os.path.join(folder_path, '*.csv'))
data = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,10,2024-03-01 17:14:06.000,0.0,1017.48,15.59,94.30,652.92,0.12
1,10,2024-03-01 17:14:47.000,0.0,1017.48,15.66,94.04,682.50,0.12
2,10,2024-03-01 17:15:47.000,0.0,1017.47,15.74,94.10,750.00,0.14
3,10,2024-03-01 17:16:47.000,0.0,1017.46,15.78,94.09,738.33,0.14
4,10,2024-03-01 17:17:47.000,0.0,1017.49,15.80,94.08,660.83,0.12
...,...,...,...,...,...,...,...,...
1375023,9,2024-10-18 14:27:02.000,0.0,1006.65,33.09,52.98,5280.83,6.71
1375024,9,2024-10-18 14:28:02.000,0.0,1006.67,33.07,53.42,4965.00,5.74
1375025,9,2024-10-18 14:29:02.000,0.0,1006.68,32.92,53.73,4709.17,5.13
1375026,9,2024-10-18 14:30:02.000,0.0,1006.70,32.92,53.55,4480.00,4.69


**合併地點與時間**

In [3]:
data['DateTime'] = pd.to_datetime(data['DateTime'])
data = data.sort_values(by=['LocationCode', 'DateTime'])
#data['Combined'] =  pd.to_datetime(data['DateTime']).dt.strftime('%Y%m%d%H%M') + data['LocationCode'].apply(lambda x: f'{x:02}')
data['DateLoc'] = data['DateTime'].dt.strftime('%Y%m%d') + data['LocationCode'].apply(lambda x : f'{x:02}')
data.to_csv('combined.csv', index=False)

data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),DateLoc
656248,1,2024-01-01 06:31:08,0.0,1016.50,17.5,86.8,25.00,0.00,2024010101
656249,1,2024-01-01 06:32:08,0.0,1016.53,17.5,86.7,28.33,0.00,2024010101
656250,1,2024-01-01 06:33:08,0.0,1016.57,17.5,86.7,32.50,0.00,2024010101
656251,1,2024-01-01 06:34:08,0.0,1016.58,17.5,86.7,39.17,0.00,2024010101
656252,1,2024-01-01 06:35:08,0.0,1016.59,17.5,86.7,45.83,0.00,2024010101
...,...,...,...,...,...,...,...,...,...
656243,17,2024-07-12 15:26:46,0.0,1004.05,36.5,65.4,12315.00,35.91,2024071217
656244,17,2024-07-12 15:27:47,0.0,1004.08,36.5,64.7,12692.50,38.23,2024071217
656245,17,2024-07-12 15:28:47,0.0,1004.10,36.5,64.5,12960.00,39.95,2024071217
656246,17,2024-07-12 15:29:47,0.0,1004.11,36.4,64.9,12751.67,38.51,2024071217


**讀取日期**

In [4]:
upload_file = 'upload.csv'
target_data = pd.read_csv(upload_file)
target_set = set()

target_data['Date'] = target_data['序號'].astype(str).str[:8]
target_data['LocationCode'] = target_data['序號'].astype(str).str[12:14]

target_data['DateLoc'] = target_data['Date'] + target_data['LocationCode']
target_set.update(target_data['DateLoc'].unique())
len(target_set)

200

**分割題目**

In [5]:
train_data = data[~data['DateLoc'].isin(target_set)]
input_data = data[data['DateLoc'].isin(target_set)]
input_data.to_csv('input.csv', index=False)
train_data.to_csv('train.csv', index=False)

**異常值處理**

**SunLight(Lux)**

In [6]:
from sklearn.model_selection import train_test_split
filtered_data = train_data[train_data['Sunlight(Lux)'] != 117758.2]
# 特徵與標籤
X = filtered_data[['Power(mW)']]  # 發電量 (特徵)
y = filtered_data['Sunlight(Lux)']  # 光照度 (目標)
# 分割訓練與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.linear_model import LinearRegression

# 建立線性回歸模型
model = LinearRegression()

# 訓練模型
model.fit( X_train,y_train )

# 獲取模型係數與截距
print(f"模型係數: {model.coef_[0]}")  # 光照度對發電量的影響係數
print(f"模型截距: {model.intercept_}")  # 發電量基線


模型係數: 55.60570241742195
模型截距: 7422.522839557063


In [8]:
from sklearn.metrics import mean_squared_error, r2_score

# 預測
y_pred = model.predict(X_test)

# 計算誤差與解釋力
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"R2: {r2}")

MSE: 99281509.20590872
R2: 0.8667737009319059


In [9]:
# 用回歸模型補充異常光照度對應的發電量
data.loc[data['Sunlight(Lux)'] == 117758.2, 'Sunlight(Lux)'] = \
    model.predict(data[data['Sunlight(Lux)'] == 117758.2][['Power(mW)']])

In [10]:
data

,LocationCode,DateTime,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),DateLoc
656248,1,2024-01-01 06:31:08,0.0,1016.50,17.5,86.8,25.00,0.00,2024010101
656249,1,2024-01-01 06:32:08,0.0,1016.53,17.5,86.7,28.33,0.00,2024010101
656250,1,2024-01-01 06:33:08,0.0,1016.57,17.5,86.7,32.50,0.00,2024010101
656251,1,2024-01-01 06:34:08,0.0,1016.58,17.5,86.7,39.17,0.00,2024010101
656252,1,2024-01-01 06:35:08,0.0,1016.59,17.5,86.7,45.83,0.00,2024010101
...,...,...,...,...,...,...,...,...,...
656243,17,2024-07-12 15:26:46,0.0,1004.05,36.5,65.4,12315.00,35.91,2024071217
656244,17,2024-07-12 15:27:47,0.0,1004.08,36.5,64.7,12692.50,38.23,2024071217
656245,17,2024-07-12 15:28:47,0.0,1004.10,36.5,64.5,12960.00,39.95,2024071217
656246,17,2024-07-12 15:29:47,0.0,1004.11,36.4,64.9,12751.67,38.51,2024071217


# Model Training

**use Lstm and pre-processed data to train our model**

In [11]:
features = ['Pressure(hpa)', 'Temperature(°C)', 
            'Humidity(%)', 'Sunlight(Lux)']
X = train_data[features]
y = train_data['Power(mW)']

In [12]:
import numpy as np

def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X.iloc[i:i+time_steps].values)
        y_seq.append(y.iloc[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 10  # 每次輸入的時間步長
X_seq, y_seq = create_sequences(X, y, time_steps)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(50, activation='relu', input_shape=(time_steps, len(features))),
    Dense(1)  # 預測單一數值
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

c:\Users\jeffi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 58s 2ms/step - loss: 204032.5938 - mae: 117.8836 - val_loss: 38983.6211 - val_mae: 81.5148
Epoch 2/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - loss: 38744.5156 - mae: 82.5797 - val_loss: 37452.9727 - val_mae: 75.7151
Epoch 3/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 37486.5195 - mae: 81.0091 - val_loss: 37192.2539 - val_mae: 82.7342
Epoch 4/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - loss: 37251.9023 - mae: 80.7778 - val_loss: 38043.2070 - val_mae: 85.5836
Epoch 5/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - loss: 37556.1484 - mae: 85.3548 - val_loss: 37486.8633 - val_mae: 83.3082
Epoch 6/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - loss: 37175.1055 - mae: 82.5074 - val_loss: 37266.1680 - val_mae: 83.6399
Epoch 7/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - loss: 36885.8945 - mae: 81.9867 - val_loss: 37213.2578 - val_mae: 81.2472
Epoch 8/10
26633/26633 ━━━━━━━━━━━━━━━━━━━━ 47s 2ms/step - loss: 39

In [ ]:
model.save('./models/trained_model.h5') 

In [21]:
predictions = model.predict(X_test)
predictions_df = pd.DataFrame(predictions, columns=['Prediction'])

# 將 DataFrame 存成 CSV
predictions_df.to_csv('predictions.csv', index=False)

print("預測結果已成功儲存到 predictions.csv！")
print(predictions_df['Prediction'].value_counts())


8323/8323 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step
預測結果已成功儲存到 predictions.csv！
Prediction
-1.889903      4
 7.330556      4
 10.773946     4
 1.543904      4
 8.344533      4
              ..
 897.405823    1
 24.781116     1
 502.301117    1
 634.479187    1
-0.054424      1
Name: count, Length: 259645, dtype: int64


In [22]:
import pandas as pd

results = pd.DataFrame({
    'Id': ['202402180900{:02d}'.format(i) for i in range(1, len(predictions) + 1)],
    'Power(mW)': predictions.flatten().round(2)
})

results.to_csv('upload(test).csv', index=False)

In [23]:
import pandas as pd
import os

# 讀取數據
df = pd.read_csv('input.csv')

# 設定儲存資料夾路徑
output_folder = './inputdata'
# 如果資料夾不存在，則創建資料夾
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 根據日期分割數據並儲存為 CSV 文件
index = 0
for date, group in df.groupby('DateLoc'):
    # 設定每個分組儲存的文件名稱
    filename = f"{date}.csv"
    file_path = os.path.join(output_folder, filename)
    
    # 儲存該分組的 CSV 文件
    group.to_csv(file_path, index=False)
    print(f"Saved: {index} {filename}")
    index +=1

Saved: 0 2024010601.csv
Saved: 1 2024011101.csv
Saved: 2 2024011701.csv
Saved: 3 2024011901.csv
Saved: 4 2024012317.csv
Saved: 5 2024012408.csv
Saved: 6 2024012502.csv
Saved: 7 2024012802.csv
Saved: 8 2024012808.csv
Saved: 9 2024012917.csv
Saved: 10 2024020408.csv
Saved: 11 2024020717.csv
Saved: 12 2024021301.csv
Saved: 13 2024021302.csv
Saved: 14 2024021413.csv
Saved: 15 2024021515.csv
Saved: 16 2024021517.csv
Saved: 17 2024021808.csv
Saved: 18 2024021815.csv
Saved: 19 2024021817.csv
Saved: 20 2024022113.csv
Saved: 21 2024022401.csv
Saved: 22 2024022417.csv
Saved: 23 2024022514.csv
Saved: 24 2024022601.csv
Saved: 25 2024022602.csv
Saved: 26 2024022714.csv
Saved: 27 2024030508.csv
Saved: 28 2024030517.csv
Saved: 29 2024030710.csv
Saved: 30 2024030909.csv
Saved: 31 2024030917.csv
Saved: 32 2024031010.csv
Saved: 33 2024031117.csv
Saved: 34 2024031409.csv
Saved: 35 2024031413.csv
Saved: 36 2024031415.csv
Saved: 37 2024031512.csv
Saved: 38 2024031610.csv
Saved: 39 2024031613.csv
Saved: 40 

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
# 假設時間步長
t =0
temp = None
# 預測函數：提取最近 time_steps 的數據
def predict_for_dataloc(dataloc, features, current_time):
    # 過濾當前時間之前的數據
    global t, temp
    df = pd.read_csv(f"inputdata/{dataloc}.csv")
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    valid_data = df[df['DateTime'] <= current_time]
    # 提取最近的 time_steps 條數據
    if(len(df)-1< t):
        return temp
    sequence = valid_data[features].tail(len(df)- t).values
    X_input = np.expand_dims(sequence, axis=0).astype(np.float32)
    predicted_power = model.predict(X_input)
    temp = predicted_power[0][0]
    return predicted_power[0][0]

# 預測函數：計算每分鐘的預測瓦數並取平均
def predict_minute_power(targetdata, dateloc, start_time, end_time, features):
    # 過濾指定的 DateLoc
    global t
    filtered_df = targetdata[targetdata['DateLoc'] == dateloc]

    # 轉換 '序號' 為時間格式
    filtered_df.loc[:, '序號'] = pd.to_datetime(filtered_df['序號'], format='%Y%m%d%H%M%S')

    # 過濾時間範圍內的數據
    filtered_df = filtered_df[(filtered_df['序號'] >= start_time) & (filtered_df['序號'] <= end_time)]
    
    # 如果範圍內有數據，計算每分鐘瓦數並取平均
    if not filtered_df.empty:
        minute_predictions = []
        current_time = start_time
        while current_time <= end_time:
            # 預測每分鐘瓦數，這裡調用 predict_for_dataloc
            predicted_power = predict_for_dataloc(dateloc, features, current_time)
            minute_predictions.append(predicted_power)
            
            # 增加 1 分鐘
            current_time += pd.Timedelta(minutes=1)
        t = t+1
        # 計算每分鐘瓦數的平均值
        final_avg_power = np.mean(minute_predictions)
    else:
        final_avg_power = 0.0

    return round(final_avg_power, 2)  # 返回最終的預測瓦數

# 計算指定時間範圍內每分鐘的瓦數並取平均
def generate_predictions(target_data, features, output_file):
    """
    根據目標數據計算每段時間的每分鐘瓦數並生成結果文件。
    """
    global t, temp
    results = []
    old = None
    for _, row in target_data.iterrows():
        # 提取序號信息
        target_date = row['Date']

        # 檢查日期是否更改
        if old == target_date:
            t += 1
        else:
            t = 0
        old = target_date
        target_time = f"{str(row['序號'])[8:10]}:{str(row['序號'])[10:12]}:00"  # 提取起始時間
        start_time = pd.Timestamp(f"{target_date} {target_time}")
        end_time = start_time + pd.Timedelta(minutes=9, seconds=59)
        # 預測該時間範圍內的瓦數
        avg_power = predict_minute_power(target_data, row['DateLoc'], start_time, end_time, features)
        # 添加結果到列表
        results.append({
            '序號': row['序號'],
            '答案': f"{avg_power:.2f}"
        })
        print(f"序號: {row['序號']}, 預測瓦數: {avg_power:.2f}")

    # 保存為 CSV 文件
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"預測結果已成功儲存到 {output_file}！")


# 執行結果生成
generate_predictions(target_data, features, 'upload.csv')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


C:\Users\jeffi\AppData\Local\Temp\ipykernel_1104\3831620408.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2024-01-17 09:00:01', '2024-01-17 09:10:01', '2024-01-17 09:20:01',
 '2024-01-17 09:30:01', '2024-01-17 09:40:01', '2024-01-17 09:50:01',
 '2024-01-17 10:00:01', '2024-01-17 10:10:01', '2024-01-17 10:20:01',
 '2024-01-17 10:30:01', '2024-01-17 10:40:01', '2024-01-17 10:50:01',
 '2024-01-17 11:00:01', '2024-01-17 11:10:01', '2024-01-17 11:20:01',
 '2024-01-17 11:30:01', '2024-01-17 11:40:01', '2024-01-17 11:50:01',
 '2024-01-17 12:00:01', '2024-01-17 12:10:01', '2024-01-17 12:20:01',
 '2024-01-17 12:30:01', '2024-01-17 12:40:01', '2024-01-17 12:50:01',
 '2024-01-17 13:00:01', '2024-01-17 13:10:01', '2024-01-17 13:20:01',
 '2024-01-17 13:30:01', '2024-01-17 13:40:01', '2024-01-17 13:50:01',
 '2024-01-17 14:00:01', '2024-01-17 14:10:01', '2024-01-17 14:20:01',
 '2024-01-17 14:30:01', '2024

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
序號: 20240117090001, 預測瓦數: 219.46
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step

C:\Users\jeffi\AppData\Local\Temp\ipykernel_1104\3831620408.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2024-01-17 09:00:01', '2024-01-17 09:10:01', '2024-01-17 09:20:01',
 '2024-01-17 09:30:01', '2024-01-17 09:40:01', '2024-01-17 09:50:01',
 '2024-01-17 10:00:01', '2024-01-17 10:10:01', '2024-01-17 10:20:01',
 '2024-01-17 10:30:01', '2024-01-17 10:40:01', '2024-01-17 10:50:01',
 '2024-01-17 11:00:01', '2024-01-17 11:10:01', '2024-01-17 11:20:01',
 '2024-01-17 11:30:01', '2024-01-17 11:40:01', '2024-01-17 11:50:01',
 '2024-01-17 12:00:01', '2024-01-17 12:10:01', '2024-01-17 12:20:01',
 '2024-01-17 12:30:01', '2024-01-17 12:40:01', '2024-01-17 12:50:01',
 '2024-01-17 13:00:01', '2024-01-17 13:10:01', '2024-01-17 13:20:01',
 '2024-01-17 13:30:01', '2024-01-17 13:40:01', '2024-01-17 13:50:01',
 '2024-01-17 14:00:01', '2024-01-17 14:10:01', '2024-01-17 14:20:01',
 '2024-01-17 14:30:01', '2024

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
序號: 20240117091001, 預測瓦數: 219.38
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


...

C:\Users\jeffi\AppData\Local\Temp\ipykernel_1104\3831620408.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2024-02-24 09:00:01', '2024-02-24 09:10:01', '2024-02-24 09:20:01',
 '2024-02-24 09:30:01', '2024-02-24 09:40:01', '2024-02-24 09:50:01',
 '2024-02-24 10:00:01', '2024-02-24 10:10:01', '2024-02-24 10:20:01',
 '2024-02-24 10:30:01', '2024-02-24 10:40:01', '2024-02-24 10:50:01',
 '2024-02-24 11:00:01', '2024-02-24 11:10:01', '2024-02-24 11:20:01',
 '2024-02-24 11:30:01', '2024-02-24 11:40:01', '2024-02-24 11:50:01',
 '2024-02-24 12:00:01', '2024-02-24 12:10:01', '2024-02-24 12:20:01',
 '2024-02-24 12:30:01', '2024-02-24 12:40:01', '2024-02-24 12:50:01',
 '2024-02-24 13:00:01', '2024-02-24 13:10:01', '2024-02-24 13:20:01',
 '2024-02-24 13:30:01', '2024-02-24 13:40:01', '2024-02-24 13:50:01',
 '2024-02-24 14:00:01', '2024-02-24 14:10:01', '2024-02-24 14:20:01',
 '2024-02-24 14:30:01', '2024

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
序號: 20240224142001, 預測瓦數: 106.49
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step

C:\Users\jeffi\AppData\Local\Temp\ipykernel_1104\3831620408.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2024-02-24 09:00:01', '2024-02-24 09:10:01', '2024-02-24 09:20:01',
 '2024-02-24 09:30:01', '2024-02-24 09:40:01', '2024-02-24 09:50:01',
 '2024-02-24 10:00:01', '2024-02-24 10:10:01', '2024-02-24 10:20:01',
 '2024-02-24 10:30:01', '2024-02-24 10:40:01', '2024-02-24 10:50:01',
 '2024-02-24 11:00:01', '2024-02-24 11:10:01', '2024-02-24 11:20:01',
 '2024-02-24 11:30:01', '2024-02-24 11:40:01', '2024-02-24 11:50:01',
 '2024-02-24 12:00:01', '2024-02-24 12:10:01', '2024-02-24 12:20:01',
 '2024-02-24 12:30:01', '2024-02-24 12:40:01', '2024-02-24 12:50:01',
 '2024-02-24 13:00:01', '2024-02-24 13:10:01', '2024-02-24 13:20:01',
 '2024-02-24 13:30:01', '2024-02-24 13:40:01', '2024-02-24 13:50:01',
 '2024-02-24 14:00:01', '2024-02-24 14:10:01', '2024-02-24 14:20:01',
 '2024-02-24 14:30:01', '2024

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


KeyboardInterrupt: 